In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import class_weight
from PIL import Image, ImageDraw, ImageFont
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
sys.path.append('/content/drive/My Drive/Data/')
from analysis_util import *
from my_classes import DataGenerator

In [4]:
with open('/content/drive/My Drive/Data/resize/X.pkl', 'rb') as fp:
  X = pickle.load(fp)

with open('/content/drive/My Drive/Data/resize/y.pkl', 'rb') as fp:
  y_classes = pickle.load(fp)

In [5]:
y = get_classes(np.array(pd.get_dummies(y_classes)))

In [6]:
val_test_size = int(len(y_classes)*.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=val_test_size, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_test_size, random_state=42)

In [7]:
dropout_rate=.3

input = keras.layers.Input(shape=(None, None, 1))

# A convolution block
x = keras.layers.Conv2D(16, kernel_size=3, strides=1, padding='same')(input)
x = keras.layers.Dropout(dropout_rate)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)

x = keras.layers.Conv2D(32, kernel_size=3, strides=1, padding='same')(x)
x = keras.layers.Dropout(dropout_rate)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)

x = keras.layers.Conv2D(64, kernel_size=3, strides=1, padding='same')(x)
x = keras.layers.Dropout(dropout_rate)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)

x = keras.layers.Conv2D(128, kernel_size=3, strides=1, padding='same')(x)
x = keras.layers.Dropout(dropout_rate)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)


# Uncomment the below line if you're using dense layers
# x = tf.keras.layers.GlobalMaxPooling2D()(x)

# Fully connected layer 1
# x = tf.keras.layers.Dropout(dropout_rate)(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Dense(units=64)(x)
# x = tf.keras.layers.Activation('relu')(x)

# Fully connected layer 1
x = keras.layers.Conv2D(1028, kernel_size=1, strides=1)(x)
x = keras.layers.Dropout(dropout_rate)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)

# Fully connected layer 2
# x = tf.keras.layers.Dropout(dropout_rate)(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Dense(units=len_classes)(x)
# predictions = tf.keras.layers.Activation('softmax')(x)

# Fully connected layer 2
x = keras.layers.Conv2D(514, kernel_size=1, strides=1)(x)
x = keras.layers.Dropout(dropout_rate)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.GlobalMaxPooling2D()(x)
predictions = keras.layers.Activation('softmax')(x)

model = keras.Model(inputs=input, outputs=predictions)
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d (Conv2D)              (None, None, None, 16)    160       
_________________________________________________________________
dropout (Dropout)            (None, None, None, 16)    0         
_________________________________________________________________
batch_normalization (BatchNo (None, None, None, 16)    64        
_________________________________________________________________
activation (Activation)      (None, None, None, 16)    0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 32)    4640      
_________________________________________________________________
dropout_1 (Dropout)          (None, None, None, 32)   

In [8]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-3), metrics=['acc'])

In [9]:
partition = {'train': X_train.index, 'validation': X_val.index}
labels = dict(zip(X.index, y))

In [10]:
training_generator = DataGenerator(partition['train'], X, labels)
validation_generator = DataGenerator(partition['validation'], X, labels)

In [12]:
model.fit_generator(training_generator.flow(), steps_per_epoch=len(X_train), epochs=1)

Instructions for updating:
Please use Model.fit, which supports generators.


ValueError: ignored